# 3.1 Colour based segmentation
This part consists of using colour based information
to segment individual images from a pumpkin field.
The segmented images should end up having a black
background with smaller white objects on top.

## Exercise 3.1.1
Annotate some pumpkins in a test image and extract information about the average pumpkin colour
in the annotated pixels. Calculate both mean value
and standard variation. Use the following two colour
spaces: RGB and CieLAB. Finally try to visualise
the distribution of colour values.

## Exercise 3.1.2
Segment the orange pumpkins from the background
using color information. Experiment with the following segmentation methods
1. inRange with RGB values
2. inRange with CieLAB values
3. Distance in RGB space to a reference colour
4. (HLS)

## Exercise 3.1.3
Choose one segmentation method to use for the rest
of the mini-project.

# 3.2 Counting objects
This part is about counting objects in segmented
images and then to generate some visual output that
will help you to debug the programs.

## Exercise 3.2.1
Count the number of orange blobs in the segmented
image.

## Exercise 3.2.2
Filter the segmented image to remove noise.

## Exercise 3.2.3
Count the number of orange blobs in the filtered image.

## Exercise 3.2.4
Mark the located pumpkins in the input image. This
step is for debugging purposes and to convince others
that you have counted the pumpkins accurately.

# 3.3 Generate an orthomosaic
This part deals with orthorectifying multiple images
of the same field into a single carthometric product.
Choose proper settings for all below processes, taking into consideration the available computing resources

## Exercise 3.3.1
Load data into Metashape.

## Exercise 3.3.2
Perform bundle adjustment (align photos) and check
results

## Exercise 3.3.3
Perform dense reconstruction

## Exercise 3.3.4
Create digital elevation model

## Exercise 3.3.5
Create orthomosaic

## Exercise 3.3.6
Limit orthomosaic to pumpkin field

# 3.4 Count in orthomosaic
Use the python package rasterio to perform operations on the orthomosaic using a tile based approach.

## Exercise 3.4.1
Create code that only loads parts of the orthomosaic.

## Exercise 3.4.2
Design tile placement incl. overlaps.

## Exercise 3.4.3
Count pumpkins in each tile.

## Exercise 3.4.4
Deal with pumpkins in the overlap, so they are only
counted once.

## Exercise 3.4.5
Determine amount of pumpkins in the entire field

# 3.5 Endnotes
Reflect on the conducted work in this miniproject.

## Exercise 3.5.1
Determine GSD and size of the image field. What is
the average number of pumpkins per area?

## Exercise 3.5.2
Reflect on whether the developed system is ready to
help a farmer with the task of estimating the number
of pumpkins in a field.